In [1]:
# import for web scraping
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import csv

    
#import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager



In [2]:
# read the names of companies from .csv (5rd col)
fortune500 = pd.read_csv('sp500_firm_execu.csv')
companies = fortune500.iloc[:,4]

# convert to list of strings and delete duplicates
companies = list(companies)
companies = list(dict.fromkeys(companies))

print(companies)





['AMERICAN AIRLINES GROUP INC', 'PINNACLE WEST CAPITAL CORP', 'ABBOTT LABORATORIES', 'ADVANCED MICRO DEVICES', 'AIR PRODUCTS & CHEMICALS INC', 'HONEYWELL INTERNATIONAL INC', 'SKYWORKS SOLUTIONS INC', 'ALCOA INC', 'HESS CORP', 'AMERICAN ELECTRIC POWER CO', 'AMERICAN EXPRESS CO', 'AFLAC INC', 'AMERICAN INTERNATIONAL GROUP', 'AMETEK INC', 'AMGEN INC', 'ANALOG DEVICES INC', 'APA CORP', 'APPLE INC', 'APPLIED MATERIALS INC', 'ARCHER-DANIELS-MIDLAND CO', 'AUTODESK INC', 'AUTOMATIC DATA PROCESSING', 'AVERY DENNISON CORP', 'BAKER HUGHES INC', 'BALL CORP', 'BANK OF NEW YORK MELLON CORP', 'BAXTER INTERNATIONAL INC', 'BECTON DICKINSON & CO', 'VERIZON COMMUNICATIONS INC', 'BERKSHIRE HATHAWAY', 'BEST BUY CO INC', 'BIO-RAD LABORATORIES INC', 'BOEING CO', 'ROBERT HALF INC', 'BRISTOL-MYERS SQUIBB CO', 'BROWN FORMAN CORP', 'CIGNA GROUP (THE)', 'CSX CORP', 'CAMPBELL SOUP CO', 'CONSTELLATION BRANDS', 'CARDINAL HEALTH INC', 'CATERPILLAR INC', 'JPMORGAN CHASE & CO', 'CHEVRON CORP', 'CHURCH & DWIGHT INC', 'C

In [3]:
#TEMP remove companies before AIR PRODUCTS & CHEMICALS INC

companies = companies[companies.index('REVVITY INC'):]
print(companies)

['REVVITY INC', 'ECOLAB INC', 'EMERSON ELECTRIC CO', 'ATMOS ENERGY CORP', 'EQUIFAX INC', 'EQT CORP', 'EXPEDITORS INTL WASH INC', 'EXXON MOBIL CORP', 'FMC CORP', 'NEXTERA ENERGY INC', 'FEDEX CORP', 'FEDERAL REALTY INVESTMENT TR', 'FIFTH THIRD BANCORP', 'REGIONS FINANCIAL CORP', 'U S BANCORP/DE-OLD', 'M&T BANK CORP', 'US BANCORP', 'FORD MOTOR CO', 'FRANKLIN RESOURCES INC', 'ARTHUR J GALLAGHER & CO', 'GENERAL DYNAMICS CORP', 'GE AEROSPACE', 'GENERAL MILLS INC', 'GENERAL MOTORS CO', 'GENUINE PARTS CO', 'GRAINGER (W W) INC', 'HALLIBURTON CO', 'L3HARRIS TECHNOLOGIES INC', 'HASBRO INC', 'WELLTOWER INC', 'KRAFT HEINZ CO', 'HERSHEY CO', 'HP INC', 'HILTON WORLDWIDE HOLDINGS', 'HOME DEPOT INC', 'HORMEL FOODS CORP', 'CENTERPOINT ENERGY INC', 'HUBBELL INC', 'HUNT (JB) TRANSPRT SVCS INC', 'HUNTINGTON BANCSHARES', 'ILLINOIS TOOL WORKS', 'TRANE TECHNOLOGIES PLC', 'INTEL CORP', 'INTL BUSINESS MACHINES CORP', 'INTL FLAVORS & FRAGRANCES', 'INTL PAPER CO', 'INTERPUBLIC GROUP OF COS', 'JACOBS SOLUTIONS INC

In [4]:

# list of time frames are strings that prefix the query, format is before:YYYY-MM-DD after:YYYY-MM-DD
# generate this dynamically
this_year = 2025
this_month = 1

time_frames = []
for i in range(24):
    year = this_year - i//12
    month = this_month - i%12
    if month <= 0:
        year -= 1
        month += 12
        
    # check if month is 30 or 31 days
    if month in [1, 3, 5, 7, 8, 10, 12]:
        days = 31
    elif month == 2:
        days = 28
    else:
        days = 30
    
    time_frames.append({'before': f'{year}-{month}-15', 'after': f'{year}-{month}-1'})
    time_frames.append({'before': f'{year}-{month}-{days}', 'after': f'{year}-{month}-15'})
    
print(' '.join(time_frames[0].values()))

    

    
    

2025-1-15 2025-1-1


In [5]:
# Set up Chrome options for headless mode
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
# chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
# chrome_options.add_argument("--window-size=1920x1080")  # Set window size
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Prevent memory issues

In [6]:
# csv_filename = 'news_data.csv'

# # take the top 5 news articles as raw text from google news for each company for each time frame of 1 month going back 2 years
# # use query prefix from time_frames

# # store the data in a pandas dataframe with columns: company, time_frame, article_text

# # initialize the dataframe
# # loop through companies
# # Set up the WebDriver
# service = Service(ChromeDriverManager().install())
# def start_driver():
#     """Initialize a fresh WebDriver instance."""
#     return webdriver.Chrome(service=service, options=chrome_options)

# driver = start_driver()
# driver.set_page_load_timeout(10)

# counter = 0

# with open(csv_filename, mode ='a', newline='') as csvfile:
#     fieldnames = ['company', 'time_frame', 'article_source', 'publish_date', 'article_text']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
#     if csvfile.tell() == 0:
#         writer.writeheader()
        
#     for company in companies:
#         print(company)
#         # loop through time_frames
#         for time_frame in time_frames:
#             # query google news
#             query = f'{company}%20before:{time_frame['before']}%20after:{time_frame['after']}'
#             url = f'https://www.google.com/search?q={query}&tbm=nws'
            
#             # use selenium to get the page source
#             try:
#                 driver.get(url)
#             except:
#                 print("Page load timed out")
#                 continue
            
#             article_a_tags = driver.find_elements(By.CSS_SELECTOR, "a.WlydOe")[:5]
            
#             article_urls = []
#             for a_tag in article_a_tags:
#                 article_urls.append(a_tag.get_attribute('href'))
                
#             articles = []
#             for url in article_urls:
#                 counter += 1
#                 if (counter >= 40): 
#                     driver.quit()
#                     driver = start_driver()
#                     driver.set_page_load_timeout(10)
#                     counter = 0
                
#                 # get the article text
#                 try:
#                     driver.get(url)
#                 except:
#                     print("Page load timed out")
#                     continue
#                 #get page source
#                 page_source = driver.page_source
#                 # clearn the page source
#                 bs4_markup = BeautifulSoup(page_source, 'html.parser')
#                 page_source = bs4_markup.get_text()
#                 # remove non-ascii characters and newlines
#                 page_source = re.sub(r'[^\x00-\x7F]+', ' ', page_source)
#                 page_source = re.sub(r'\n', ' ', page_source)
                
#                 #remove commas
#                 page_source = re.sub(r',', '', page_source)
                
#                 # get article publish date from last modified http header
#                 publish_date = driver.execute_script("return document.lastModified")
#                 print(publish_date)
                
                
                
            
#                 articles.append({'url': url, 'text': page_source, 'publish_date': publish_date})

                
#             # store the data
#             for article in articles:
#                 writer.writerow({'company': company, 'time_frame': ' '.join(time_frame.values()), 'article_source': article['url'], 'publish_date': article['publish_date'], 'article_text': article['text']})
#                 print(f'{company} {time_frame} {article["url"]}')
    
# driver.quit()    
        

In [ ]:
import asyncio
from pyppeteer import launch

CHROME_PATH = r"C:\Users\Mahyar\Downloads\chrome-win\chrome-win\chrome.exe"
    

csv_filename = 'data/news_data.csv'

with open(csv_filename, mode ='a', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = ['company', 'time_frame', 'article_source', 'publish_date', 'article_text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    if csvfile.tell() == 0:
        writer.writeheader()
        
    
        
    for company in companies:
        browser = await launch(headless=True, executablePath=CHROME_PATH, args=["--no-sandbox", "--disable-setuid-sandbox"])  # Run in headless mode
        print(company)
        # loop through time_frames
        for time_frame in time_frames:
            # query google news
            query = f'{company}%20before:{time_frame['before']}%20after:{time_frame['after']}'
            url = f'https://www.google.com/search?q={query}&tbm=nws'

            
            page = await browser.newPage()
            await page.goto(url, {"waitUntil": "networkidle2"})  # Wait for full page load
            

            # Extract all links with the given class
            try :
                article_urls = await page.evaluate(f'''
                    () => {{
                        return Array.from(document.querySelectorAll("a.WlydOe"))
                                    .map(a => a.href);
                    }}
                ''')
            except:
                print("could not find article urls")
                continue
                
            articles = []
            for url in article_urls:
                # get the article text
                try:
                    page_source = await page.content()  # Get HTML source
                except:
                    print("Page load timed out")
                    continue
                # clearn the page source
                bs4_markup = BeautifulSoup(page_source, 'html.parser')
                page_source = bs4_markup.get_text()
                # remove non-ascii characters and newlines
                page_source = re.sub(r'[^\x00-\x7F]+', ' ', page_source)
                page_source = re.sub(r'\n', ' ', page_source)
                
                #remove commas
                page_source = re.sub(r',', '', page_source)
                
                # get article publish date from last modified http header with puppeteer
                publish_date =  await page.evaluate(f'''
                    () => {{
                        return document.lastModified;
                    }}
                ''')
                print(publish_date)
                
                
                
            
                articles.append({'url': url, 'text': page_source, 'publish_date': publish_date})

            # store the data
            for article in articles:
                writer.writerow({'company': company, 'time_frame': ' '.join(time_frame.values()), 'article_source': article['url'], 'publish_date': article['publish_date'], 'article_text': article['text']})
                print(f'{company} {time_frame} {article["url"]}')


    await browser.close()

REVVITY INC
01/31/2025 06:47:43
01/31/2025 06:47:43
01/31/2025 06:47:43
01/31/2025 06:47:44
01/31/2025 06:47:44
01/31/2025 06:47:44
01/31/2025 06:47:44
01/31/2025 06:47:44
01/31/2025 06:47:44
01/31/2025 06:47:44
REVVITY INC {'before': '2025-1-15', 'after': '2025-1-1'} https://www.businesswire.com/news/home/20250112188630/en/Revvity-to-Hold-Earnings-Call-on-Friday-January-31-2025-Provides-Update-on-Financial-Performance
REVVITY INC {'before': '2025-1-15', 'after': '2025-1-1'} https://www.marketbeat.com/instant-alerts/tajinder-s-vohra-sells-5492-shares-of-revvity-inc-nyservty-stock-2025-01-08/
REVVITY INC {'before': '2025-1-15', 'after': '2025-1-1'} https://simplywall.st/stocks/us/pharmaceuticals-biotech/nyse-rvty/revvity/news/shareholders-should-be-pleased-with-revvity-incs-nyservty-pr
REVVITY INC {'before': '2025-1-15', 'after': '2025-1-1'} https://www.investing.com/news/sec-filings/revvity-secures-new-15-billion-credit-facility-93CH-3801471
REVVITY INC {'before': '2025-1-15', 'after':